In [114]:
import os
import time
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [115]:
DOWNLOADS_PATH = 'C:\\Users\\riger\\Downloads'
SCRIPT_PATH = os.getcwd()
COMBINED_CSV_PATH = os.path.join(SCRIPT_PATH, 'combined.csv')
BASE_URL = 'https://www.nhl.com/stats/teams?aggregate=0&report=daysbetweengames&reportType=game&dateFrom=2024-10-04&dateTo=2025-01-06&gameType=2&sort=a_teamFullName,daysRest&page={}&pageSize=100'

In [116]:
# Setup Chrome options for automatic download
chrome_options = Options()
chrome_options.add_experimental_option('prefs', {
    'download.default_directory': DOWNLOADS_PATH,
    'download.prompt_for_download': False,
    'download.directory_upgrade': True,
    'safebrowsing.enabled': True
})

In [117]:
service = Service('chromedriver.exe')
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.maximize_window()

try:
    combined_data = pd.DataFrame()

    for page in range(0, 13):
        url = BASE_URL.format(page)
        driver.get(url)

        # Wait for the export link to be present and clickable
        export_link = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/main/div/div/div/div[2]/div/div[2]/div/main/div[2]/h4/a'))
        )

        # Scroll to the element
        driver.execute_script("arguments[0].scrollIntoView(true);", export_link)
        export_link.click()
        time.sleep(5)  # Wait for the file to download

        # Process the downloaded file
        downloaded_file = os.path.join(DOWNLOADS_PATH, 'Days between Games.xlsx')
        if os.path.exists(downloaded_file):
            data = pd.read_excel(downloaded_file)
            combined_data = pd.concat([combined_data, data], ignore_index=True)
            os.remove(downloaded_file)  # Remove the downloaded file after processing

    # Save the combined data as a CSV file
    if os.path.exists('Days between Games.xlsx'):
        os.remove('Days between Games.xlsx')
    combined_data.to_csv(COMBINED_CSV_PATH, index=False)
    print(f"Combined CSV saved at: {COMBINED_CSV_PATH}")

finally:
    driver.quit()

Combined CSV saved at: C:\Users\riger\Desktop\dataScraper\NHL_data_scraper\combined.csv


In [118]:
temp_df = pd.read_csv('combined.csv')

In [119]:
temp_df = temp_df.sort_values(by="Game Date")

In [121]:
hockey_ref_url = 'https://www.hockey-reference.com/leagues/NHL_2025_games.html'
hockey_ref_dfs = pd.read_html(hockey_ref_url)
hockey_ref_dfs[0].to_csv('nhl_ref_data.csv', index=False)

In [122]:
nhl_ref_data = pd.read_csv('nhl_ref_data.csv')

In [124]:
nhl_data = pd.read_csv('combined.csv')
nhl_ref_data = pd.read_csv('nhl_ref_data.csv')

def add_home_column(final_df, temp_df):
    # step 1: loop thorugh the temp df
    # step 2: get display a second df from the main where the dates match the temp df
    # step 3: loop through the second df until 'Team' matches 'Home' and 'Date' matches 'Date' in the temp df
    # step 4: add the 'Home' column to the main df for that row with the value of 1
    for i in range(len(temp_df)):  # Iterate over rows in temp_df
        temp_date = temp_df['Date'][i]
        new_df = final_df[final_df['Game Date'] == temp_date]
        
        for j in range(len(new_df)):
            if new_df['Team'].iloc[j] == temp_df['Home'].iloc[i]:
                if 'Venue' not in final_df.columns:
                    final_df['Venue'] = 'Away'
                
                # Update the specific row in the original df
                final_df.loc[new_df.index[j], 'Venue'] = 'Home'
    
    return final_df

In [125]:
final_nhl_data = add_home_column(nhl_data, nhl_ref_data)
final_nhl_data = final_nhl_data.dropna()

In [126]:
def add_time_column(final_df, temp_df):
    # Ensure 'Time' column exists in final_df
    if 'Time' not in final_df.columns:
        final_df['Time'] = None

    # Iterate over rows in temp_df
    for i in range(len(temp_df)):
        temp_time = temp_df['Time'].iloc[i]
        temp_date = temp_df['Date'].iloc[i]
        temp_home = temp_df['Home'].iloc[i]
        temp_visitor = temp_df['Visitor'].iloc[i]

        # Filter final_df for matching date
        matching_rows = final_df[final_df['Game Date'] == temp_date]

        for j in range(len(matching_rows)):
            team = matching_rows['Team'].iloc[j]

            # Check if the team matches either Home or Visitor in temp_df
            if team == temp_home or team == temp_visitor:
                # Update the 'Time' column in the original final_df
                final_df.loc[matching_rows.index[j], 'Time'] = temp_time

    return final_df

In [127]:
final_nhl_data = add_time_column(nhl_data, nhl_ref_data)

In [128]:
final_nhl_data['Time'] = final_nhl_data['Time'].apply(
    lambda x: pd.to_datetime(x, format='%I:%M %p').strftime('%H:%M') if pd.notnull(x) else None
)

In [129]:
final_nhl_data = final_nhl_data.sort_values(by="Game Date")

In [131]:
final_nhl_data.rename(columns={'OT':'OT Losses','GD/GP':'Net Goals','Shots/GP':'Shots For','SA/GP':'Shots Against','SD/GP':'Shot Diff'}, inplace=True)
final_nhl_data['Result'] = final_nhl_data.apply(lambda row: 'W' if row['Net Goals'] > 0 else 'L', axis=1)

In [132]:
final_nhl_data.head()

,Team,Opp Team,Game Date,Days Btwn Games,GP,W,L,T,OT Losses,P,...,Shots Against,Shot Diff,PP Opp/GP,TS/GP,PP%,PK%,FOW%,Venue,Time,Result
81,Buffalo Sabres,NJD,2024-10-04,4,1,0,1,--,0,0,...,23,8,4,2,0,100,57.6,Home,13:00,L
639,New Jersey Devils,BUF,2024-10-04,4,1,1,0,--,0,2,...,31,-8,2,4,0,100,42.4,Away,13:00,W
117,Buffalo Sabres,NJD,2024-10-05,0,1,0,1,--,0,0,...,37,-19,2,4,0,75,35.3,Away,10:00,L
675,New Jersey Devils,BUF,2024-10-05,0,1,1,0,--,0,2,...,18,19,4,2,25,100,64.7,Home,10:00,W
440,Florida Panthers,BOS,2024-10-08,4,1,1,0,--,0,2,...,28,7,6,4,0,75,51.6,Home,19:00,W


In [133]:
nhl_team_abbr = {
    'ANA' : 'Anaheim Ducks',
    'BOS' : 'Boston Bruins',
    'BUF' : 'Buffalo Sabres',
    'CAR' : 'Carolina Hurricanes',
    'CBJ' : 'Columbus Blue Jackets',
    'CGY' : 'Calgary Flames',
    'CHI' : 'Chicago Blackhawks',
    'COL' : 'Colorado Avalanche',
    'DAL' : 'Dallas Stars',
    'DET' : 'Detroit Red Wings',
    'EDM' : 'Edmonton Oilers',
    'FLA' : 'Florida Panthers',
    'LAK' : 'Los Angeles Kings',
    'MIN' : 'Minnesota Wild',
    'MTL' : 'Montreal Canadiens',
    'NJD' : 'New Jersey Devils',
    'NSH' : 'Nashville Predators',
    'NYI' : 'New York Islanders',
    'NYR' : 'New York Rangers',
    'OTT' : 'Ottawa Senators',
    'PHI' : 'Philadelphia Flyers',
    'PIT' : 'Pittsburgh Penguins',
    'SJS' : 'San Jose Sharks',
    'SEA' : 'Seattle Kraken',
    'STL' : 'St. Louis Blues',
    'TBL' : 'Tampa Bay Lightning',
    'TOR' : 'Toronto Maple Leafs',
    'UTA' : 'Utah Hockey Club',
    'VAN' : 'Vancouver Canucks',
    'VGK' : 'Vegas Golden Knights',
    'WPG' : 'Winnipeg Jets',
    'WSH' : 'Washington Capitals'
}

In [134]:
final_nhl_data['Opp Team'] = final_nhl_data['Opp Team'].apply(lambda abbr: nhl_team_abbr.get(abbr, abbr))

In [135]:
final_nhl_data.drop(columns=['W', 'L', 'T', 'GP', 'OT Losses', 'P', 'P%'], inplace=True)

In [136]:
# Step 4: Remove the other temp csv files
os.remove('nhl_ref_data.csv')
os.remove('combined.csv')

In [137]:
final_nhl_data.head()

,Team,Opp Team,Game Date,Days Btwn Games,GF/GP,GA/GP,Net Goals,Shots For,Shots Against,Shot Diff,PP Opp/GP,TS/GP,PP%,PK%,FOW%,Venue,Time,Result
81,Buffalo Sabres,New Jersey Devils,2024-10-04,4,1,4,-3,31,23,8,4,2,0,100,57.6,Home,13:00,L
639,New Jersey Devils,Buffalo Sabres,2024-10-04,4,4,1,3,23,31,-8,2,4,0,100,42.4,Away,13:00,W
117,Buffalo Sabres,New Jersey Devils,2024-10-05,0,1,3,-2,18,37,-19,2,4,0,75,35.3,Away,10:00,L
675,New Jersey Devils,Buffalo Sabres,2024-10-05,0,3,1,2,37,18,19,4,2,25,100,64.7,Home,10:00,W
440,Florida Panthers,Boston Bruins,2024-10-08,4,6,4,2,35,28,7,6,4,0,75,51.6,Home,19:00,W


In [138]:
final_nhl_data.columns = [x.lower() for x in final_nhl_data.columns]

In [139]:
final_nhl_data.head()

,team,opp team,game date,days btwn games,gf/gp,ga/gp,net goals,shots for,shots against,shot diff,pp opp/gp,ts/gp,pp%,pk%,fow%,venue,time,result
81,Buffalo Sabres,New Jersey Devils,2024-10-04,4,1,4,-3,31,23,8,4,2,0,100,57.6,Home,13:00,L
639,New Jersey Devils,Buffalo Sabres,2024-10-04,4,4,1,3,23,31,-8,2,4,0,100,42.4,Away,13:00,W
117,Buffalo Sabres,New Jersey Devils,2024-10-05,0,1,3,-2,18,37,-19,2,4,0,75,35.3,Away,10:00,L
675,New Jersey Devils,Buffalo Sabres,2024-10-05,0,3,1,2,37,18,19,4,2,25,100,64.7,Home,10:00,W
440,Florida Panthers,Boston Bruins,2024-10-08,4,6,4,2,35,28,7,6,4,0,75,51.6,Home,19:00,W


In [140]:
final_nhl_data = final_nhl_data.rename(columns={"game date": "date", "opp team": "opponent", "days btwn games": "rest days"})

In [141]:
final_nhl_data.head()

,team,opponent,date,rest days,gf/gp,ga/gp,net goals,shots for,shots against,shot diff,pp opp/gp,ts/gp,pp%,pk%,fow%,venue,time,result
81,Buffalo Sabres,New Jersey Devils,2024-10-04,4,1,4,-3,31,23,8,4,2,0,100,57.6,Home,13:00,L
639,New Jersey Devils,Buffalo Sabres,2024-10-04,4,4,1,3,23,31,-8,2,4,0,100,42.4,Away,13:00,W
117,Buffalo Sabres,New Jersey Devils,2024-10-05,0,1,3,-2,18,37,-19,2,4,0,75,35.3,Away,10:00,L
675,New Jersey Devils,Buffalo Sabres,2024-10-05,0,3,1,2,37,18,19,4,2,25,100,64.7,Home,10:00,W
440,Florida Panthers,Boston Bruins,2024-10-08,4,6,4,2,35,28,7,6,4,0,75,51.6,Home,19:00,W


In [142]:
final_nhl_data.columns

Index(['team', 'opponent', 'date', 'rest days', 'gf/gp', 'ga/gp', 'net goals',
       'shots for', 'shots against', 'shot diff', 'pp opp/gp', 'ts/gp', 'pp%',
       'pk%', 'fow%', 'venue', 'time', 'result'],
      dtype='object')

In [143]:
new_order = ['date', 'time', 'venue', 'rest days', 'result', 'gf/gp', 'ga/gp', 'net goals', 'opponent', 'shots for', 'shots against', 'shot diff', 'pp opp/gp', 'ts/gp', 'pp%',
       'pk%', 'fow%', 'team']

In [144]:
final_nhl_data = final_nhl_data[new_order]

In [145]:
final_nhl_data

,date,time,venue,rest days,result,gf/gp,ga/gp,net goals,opponent,shots for,shots against,shot diff,pp opp/gp,ts/gp,pp%,pk%,fow%,team
81,2024-10-04,13:00,Home,4,L,1,4,-3,New Jersey Devils,31,23,8,4,2,0,100,57.6,Buffalo Sabres
639,2024-10-04,13:00,Away,4,W,4,1,3,Buffalo Sabres,23,31,-8,2,4,0,100,42.4,New Jersey Devils
117,2024-10-05,10:00,Away,0,L,1,3,-2,New Jersey Devils,18,37,-19,2,4,0,75,35.3,Buffalo Sabres
675,2024-10-05,10:00,Home,0,W,3,1,2,Buffalo Sabres,37,18,19,4,2,25,100,64.7,New Jersey Devils
440,2024-10-08,19:00,Home,4,W,6,4,2,Boston Bruins,35,28,7,6,4,0,75,51.6,Florida Panthers
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
940,2025-01-06,22:00,Home,1,L,2,3,-1,New Jersey Devils,24,29,-5,1,0,100.0,--,47.4,Seattle Kraken
272,2025-01-06,21:00,Home,1,W,3,1,2,Florida Panthers,34,27,7,4,3,0,100,50.9,Colorado Avalanche
1224,2025-01-06,19:00,Away,1,L,3,3,0,Buffalo Sabres,28,27,1,2,3,50,66.7,54.1,Washington Capitals
109,2025-01-06,19:00,Home,1,L,3,3,0,Washington Capitals,27,28,-1,3,2,33.3,50,45.9,Buffalo Sabres


In [146]:
final_nhl_data.to_csv('final_nhl_data.csv', index=False)